<a href="https://colab.research.google.com/github/divya-medarametla/CMPE-255-Credit_Hazard-Forecasting/blob/main/Credit_hazard_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np



credit_approval = pd.read_csv("credit_risk_dataset.csv", header = None)
credit_approval.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
2,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.1,N,2
3,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
4,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2


In [ ]:
import os

In [ ]:
DATASET_DIR='dataset/'
DATASET_FILENAME='credit_risk_dataset.csv'

In [ ]:
credit_approval.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,32582,32582,32582,31687,32582,32582,32582,29466,32582,32582,32582,32582
unique,59,4296,5,37,7,8,754,349,3,78,3,30
top,23,60000,RENT,0.0,EDUCATION,A,10000,7.51,0,0.1,N,2
freq,3889,1046,16446,4105,6453,10777,2664,756,25473,1533,26836,5965


In [ ]:
credit_approval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32582 entries, 0 to 32581
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32582 non-null  object
 1   1       32582 non-null  object
 2   2       32582 non-null  object
 3   3       31687 non-null  object
 4   4       32582 non-null  object
 5   5       32582 non-null  object
 6   6       32582 non-null  object
 7   7       29466 non-null  object
 8   8       32582 non-null  object
 9   9       32582 non-null  object
 10  10      32582 non-null  object
 11  11      32582 non-null  object
dtypes: object(12)
memory usage: 3.0+ MB


In [ ]:
def check_missing_columns(df_in):
 
    missing_columns_list=[]
    print("\n[INFO] Checking missing values in features\n")
    for col in df_in.columns:
        print("[INFO] Feature Selected:",col)
        missing_data_count=df_in[col].isna().sum()
        print("[INFO] Total nan's:",missing_data_count)
        
        values_with_palce_holder=len(df_in[df_in[col]=='?'])
        print("[INFO] Total '?' :",values_with_palce_holder)
        
        
        missing_data_count=missing_data_count+values_with_palce_holder
        if missing_data_count>0:
            print("[INFO] Total missing data in "+col+":"+str(missing_data_count))
            missing_columns_list.append(col)
            print("#"*100)
        else:
            print("[INFO] No missing data for col:",col)
            print("#"*100)
        return missing_columns_list

In [ ]:
missing_columns=check_missing_columns(credit_approval)


[INFO] Checking missing values in features

[INFO] Feature Selected: 0
[INFO] Total nan's: 0
[INFO] Total '?' : 0
[INFO] No missing data for col: 0
####################################################################################################


In [ ]:
def impute_missing_values(df_in,features_with_missing_values):
    
    

    imputed_value_dict={}
    print("\n[INFO] Imputing missing values\n")
    print("#"*10)
    for col in features_with_missing_values:
        print("[INFO] Imputing :",col)
        df_in[col]=df_in[col].replace('?',np.nan)
        df_in[col]=df_in[col].astype(float)
        mean_value=np.mean(df_in[col])
        df_in[col]=df_in[col].fillna(mean_value)
        print("[INFO] Imputing with Mean:",mean_value)
        imputed_value_dict[col]=mean_value
        
    return df_in,imputed_value_dict

In [ ]:
missing_columns=check_missing_columns(credit_approval)


[INFO] Checking missing values in features

[INFO] Feature Selected: 0
[INFO] Total nan's: 0
[INFO] Total '?' : 0
[INFO] No missing data for col: 0
####################################################################################################


In [ ]:
credit_approval

,0,1,2,3,4,5,6,7,8,9,10,11
0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
2,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.1,N,2
3,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
4,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
...,...,...,...,...,...,...,...,...,...,...,...,...
32577,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32578,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32579,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32580,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.1,N,26


In [ ]:
df_temp

,0,1,2,3,4,5,6,7,8,9,10,11
0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
2,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.1,N,2
3,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
4,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
...,...,...,...,...,...,...,...,...,...,...,...,...
32577,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32578,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32579,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32580,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.1,N,26


In [ ]:
credit_approval.isna().sum()

0        0
1        0
2        0
3      895
4        0
5        0
6        0
7     3116
8        0
9        0
10       0
11       0
dtype: int64

In [ ]:
credit_approval.shape

(32582, 12)

In [ ]:
def read_dataset(file_path):
  
    data_flag=False
    data=[]
    features=[]
    with open(file_path) as f:
        all_lines=f.readlines()
        for line in all_lines:
            if '@ATTRIBUTE' in line:
                _,feature_name,_=line.split(' ')
                features.append(feature_name)
            if data_flag:
                line=line.replace("\n",'')
                data.append(line.split(','))
            if '@DATA' in line:
                data_flag=True   
    df_data=pd.DataFrame(data=data,columns=features)
    return df_data


In [ ]:

import numpy as np
credit_approval = credit_approval.replace("?", np.NaN)

In [ ]:

print(credit_approval[0].value_counts())
credit_approval[0] = credit_approval[0].fillna(credit_approval[0].value_counts().index[0])
credit_approval[0].tail(5)

23            3889
22            3633
24            3549
25            3037
26            2477
27            2138
28            1854
29            1687
30            1316
21            1229
31            1142
32             964
33             856
34             709
35             620
36             548
37             478
38             373
39             302
40             271
41             241
42             188
43             164
44             141
45             108
47              94
46              94
48              75
50              52
49              49
51              39
52              36
53              30
54              24
55              20
58              19
56              15
60              15
20              15
57              15
66               9
61               9
65               9
62               7
64               7
70               7
59               5
69               5
144              3
73               3
63               3
123              2
80          

32577    57
32578    54
32579    65
32580    56
32581    66
Name: 0, dtype: object

In [ ]:

print(credit_approval[3].value_counts())
credit_approval[3] = credit_approval[3].fillna(credit_approval[3].value_counts().index[0])
credit_approval[3].tail(5)

0.0                  5000
2.0                  3849
3.0                  3456
5.0                  2946
1.0                  2915
4.0                  2874
6.0                  2666
7.0                  2196
8.0                  1687
9.0                  1367
11.0                  740
10.0                  696
12.0                  575
13.0                  426
14.0                  335
15.0                  238
16.0                  165
17.0                  129
18.0                  104
19.0                   64
20.0                   42
21.0                   38
22.0                   19
24.0                   10
23.0                   10
25.0                    8
26.0                    6
27.0                    5
31.0                    4
28.0                    3
30.0                    2
123.0                   2
41.0                    1
34.0                    1
29.0                    1
38.0                    1
person_emp_length       1
Name: 3, dtype: int64


32577    1.0
32578    4.0
32579    3.0
32580    5.0
32581    2.0
Name: 3, dtype: object

In [ ]:

print(credit_approval[4].value_counts())
credit_approval[4] = credit_approval[4].fillna(credit_approval[4].value_counts().index[0])
credit_approval[4].tail(5)

EDUCATION            6453
MEDICAL              6071
VENTURE              5719
PERSONAL             5521
DEBTCONSOLIDATION    5212
HOMEIMPROVEMENT      3605
loan_intent             1
Name: 4, dtype: int64


32577           PERSONAL
32578           PERSONAL
32579    HOMEIMPROVEMENT
32580           PERSONAL
32581            MEDICAL
Name: 4, dtype: object

In [ ]:

print(credit_approval[5].value_counts())
credit_approval[5] = credit_approval[5].fillna(credit_approval[5].value_counts().index[0])
print(credit_approval[5].isnull().values.sum())

A             10777
B             10451
C              6458
D              3626
E               964
F               241
G                64
loan_grade        1
Name: 5, dtype: int64
0


In [ ]:

print(credit_approval[6].value_counts())
print(credit_approval[6].isnull().values.sum())
credit_approval[6] = credit_approval[6].fillna(credit_approval[6].value_counts().index[0])
credit_approval[6].tail(5)

10000    2664
5000     2043
12000    1809
6000     1801
15000    1502
         ... 
29175       1
27300       1
17325       1
9625        1
29850       1
Name: 6, Length: 754, dtype: int64
0


32577     5800
32578    17625
32579    35000
32580    15000
32581     6475
Name: 6, dtype: object

In [ ]:

from sklearn.preprocessing import MinMaxScaler
def preprocess(data):
  scaler = MinMaxScaler(feature_range=(0, 1))
  return scaler.fit_transform(data)

In [7]:

credit_approval.to_csv('crx_clean.data')
credit_approval.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
2,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.1,N,2
3,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
4,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
5,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
6,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2
7,26,77100,RENT,8.0,EDUCATION,B,35000,12.42,1,0.45,N,3
8,24,78956,RENT,5.0,MEDICAL,B,35000,11.11,1,0.44,N,4
9,24,83000,RENT,8.0,PERSONAL,A,35000,8.9,1,0.42,N,2


In [10]:
credit_approval.to_csv('crx_clean.data')
credit_approval.head(6)

,0,1,2,3,4,5,6,7,8,9,10,11
0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
2,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.1,N,2
3,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
4,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
5,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
6,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2
7,26,77100,RENT,8.0,EDUCATION,B,35000,12.42,1,0.45,N,3
8,24,78956,RENT,5.0,MEDICAL,B,35000,11.11,1,0.44,N,4
9,24,83000,RENT,8.0,PERSONAL,A,35000,8.9,1,0.42,N,2
